# ═══════════════════════════════════════════════════════════════════════════════
# 🍒 DECISION POINT: SELECT AUDIENCE(S) TO INCLUDE
# ═══════════════════════════════════════════════════════════════════════════════

Configure which role(s) to include in the experiment below.

**Available Role Categories (Granular):**
| Code | Role | Typical N |
|------|------|----------|
| `FT_Student` | Full-time Student | ~150 |
| `PT_Student` | Part-time Student | ~46 |
| `IC` | Individual Contributor | ~100 |
| `Manager` | Manager | ~90 |
| `Executive` | Executive/Leader | ~40 |
| `Freelancer` | Freelancer/Self-Employed | ~40 |
| `Unemployed` | Not Currently Employed | ~21 |

**Pre-defined Population Groups:**
| Group | Includes |
|-------|----------|
| `Academic` | FT_Student + PT_Student |
| `Professional` | IC + Freelancer + Unemployed |
| `Leader` | Manager + Executive |
| `ALL` | All roles above (~487 total) |

In [1]:
# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║  🎯 DECISION POINT: EDIT THIS TO SELECT ROLE(S)                              ║
# ╚══════════════════════════════════════════════════════════════════════════════╝

# ═══════════════════════════════════════════════════════════════════════════════
# OPTION 1: Process ALL roles (default - maximum power for cherry-picking)
# ═══════════════════════════════════════════════════════════════════════════════
SELECTED_ROLES = ['ALL']

# ═══════════════════════════════════════════════════════════════════════════════
# OPTION 2: Population Groups (coarse)
# ═══════════════════════════════════════════════════════════════════════════════
# SELECTED_ROLES = ['Academic']                    # FT_Student + PT_Student (~196)
# SELECTED_ROLES = ['Professional']                # IC + Freelancer + Unemployed (~161)
# SELECTED_ROLES = ['Leader']                      # Manager + Executive (~130)
# SELECTED_ROLES = ['Academic', 'Professional']    # Exclude leaders

# ═══════════════════════════════════════════════════════════════════════════════
# OPTION 3: Granular Role Selection
# ═══════════════════════════════════════════════════════════════════════════════
# SELECTED_ROLES = ['FT_Student']                  # Full-time students only
# SELECTED_ROLES = ['PT_Student']                  # Part-time students only
# SELECTED_ROLES = ['IC']                          # Individual contributors only
# SELECTED_ROLES = ['Manager']                     # Managers only
# SELECTED_ROLES = ['Executive']                   # Executives only
# SELECTED_ROLES = ['Freelancer']                  # Freelancers/Self-employed only
# SELECTED_ROLES = ['Unemployed']                  # Not currently employed only

# ═══════════════════════════════════════════════════════════════════════════════
# OPTION 4: Custom Combinations
# ═══════════════════════════════════════════════════════════════════════════════
# SELECTED_ROLES = ['FT_Student', 'IC']            # Students + ICs only
# SELECTED_ROLES = ['Manager', 'Executive']        # Leaders only (same as 'Leader')
# SELECTED_ROLES = ['FT_Student', 'Manager']       # Compare students vs managers
# SELECTED_ROLES = ['IC', 'Freelancer']            # Non-manager professionals
# SELECTED_ROLES = ['Academic', 'Manager']         # Students + Managers

print(f"🍒 Selected roles: {SELECTED_ROLES}")

🍒 Selected roles: ['ALL']


In [2]:
# ═══════════════════════════════════════════════════════════════════════════════
# SECTION 0: IMPORTS AND CONFIGURATION
# ═══════════════════════════════════════════════════════════════════════════════

import sys
import os
import json
import warnings
from pathlib import Path

# Prevent OpenMP conflicts
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')

# Configuration
RANDOM_SEED = 67
np.random.seed(RANDOM_SEED)

# Create output directories
Path('data').mkdir(exist_ok=True)
Path('plots').mkdir(exist_ok=True)
Path('tables').mkdir(exist_ok=True)

print("="*70)
print("🍒 AIRS EXPERIMENT DATA PREPARATION")
print("="*70)
print(f"Random seed: {RANDOM_SEED}")
print(f"Output: Full sample (no dev/holdout split)")
print("="*70)

🍒 AIRS EXPERIMENT DATA PREPARATION
Random seed: 67
Output: Full sample (no dev/holdout split)


In [3]:
# ═══════════════════════════════════════════════════════════════════════════════
# Role Group Definitions
# ═══════════════════════════════════════════════════════════════════════════════

ROLE_GROUPS = {
    'ALL': ['FT_Student', 'PT_Student', 'IC', 'Manager', 'Executive', 'Freelancer', 'Unemployed'],
    'Academic': ['FT_Student', 'PT_Student'],
    'Professional': ['IC', 'Freelancer', 'Unemployed'],
    'Leader': ['Manager', 'Executive'],
}

VALID_GRANULAR_ROLES = ['FT_Student', 'PT_Student', 'IC', 'Manager', 'Executive', 'Freelancer', 'Unemployed']
VALID_GROUPS = list(ROLE_GROUPS.keys())

# Expand selection to granular roles
ROLES_TO_INCLUDE = []
for role in SELECTED_ROLES:
    if role in ROLE_GROUPS:
        ROLES_TO_INCLUDE.extend(ROLE_GROUPS[role])
    elif role in VALID_GRANULAR_ROLES:
        ROLES_TO_INCLUDE.append(role)
    else:
        raise ValueError(f"Invalid role: {role}. Valid options: {VALID_GROUPS + VALID_GRANULAR_ROLES}")

# Remove duplicates while preserving order
ROLES_TO_INCLUDE = list(dict.fromkeys(ROLES_TO_INCLUDE))

print("🍒 AUDIENCE CONFIGURATION")
print("="*70)
print(f"Selected: {SELECTED_ROLES}")
print(f"Granular roles to include: {ROLES_TO_INCLUDE}")
print("="*70)

🍒 AUDIENCE CONFIGURATION
Selected: ['ALL']
Granular roles to include: ['FT_Student', 'PT_Student', 'IC', 'Manager', 'Executive', 'Freelancer', 'Unemployed']


## 1. Load Clean Data from Main Pipeline

In [4]:
# ═══════════════════════════════════════════════════════════════════════════════
# SECTION 1: LOAD CLEAN DATA
# ═══════════════════════════════════════════════════════════════════════════════

# Load from main data folder (processed by 00_Create_Split_Samples.ipynb)
df_full = pd.read_csv('../data/AIRS_clean.csv')

print("="*70)
print("DATA LOADED")
print("="*70)
print(f"Total rows: {len(df_full)}")
print(f"Columns: {len(df_full.columns)}")
print("="*70)

DATA LOADED
Total rows: 511
Columns: 45


In [5]:
# ═══════════════════════════════════════════════════════════════════════════════
# SECTION 2: APPLY ROLE FILTER
# ═══════════════════════════════════════════════════════════════════════════════

# Filter to valid populations (exclude 'Other')
df_valid = df_full[df_full['Population'].notna()].copy()

# Apply granular role filter
df = df_valid[df_valid['Role_Category'].isin(ROLES_TO_INCLUDE)].copy()

print("="*70)
print("🍒 AUDIENCE FILTERING")
print("="*70)
print(f"Total responses: {len(df_full)}")
print(f"Valid populations (excl. 'Other'): {len(df_valid)}")
print(f"After role filter: {len(df)}")
print(f"Excluded: {len(df_valid) - len(df)}")
print("="*70)

# Show role breakdown
print("\n🍒 Role breakdown in experiment sample:")
role_counts = df['Role_Category'].value_counts()
for role, n in role_counts.items():
    marker = "✓" if role in ROLES_TO_INCLUDE else "✗"
    print(f"   {marker} {role}: N={n}")

# Show population breakdown
print("\n📊 Population breakdown:")
pop_counts = df['Population'].value_counts()
for pop, n in pop_counts.items():
    print(f"   {pop}: N={n} ({n/len(df)*100:.1f}%)")

🍒 AUDIENCE FILTERING
Total responses: 511
Valid populations (excl. 'Other'): 487
After role filter: 487
Excluded: 0

🍒 Role breakdown in experiment sample:
   ✓ FT_Student: N=180
   ✓ IC: N=113
   ✓ Manager: N=74
   ✓ Executive: N=56
   ✓ Freelancer: N=31
   ✓ Unemployed: N=17
   ✓ PT_Student: N=16

📊 Population breakdown:
   Academic: N=196 (40.2%)
   Professional: N=161 (33.1%)
   Leader: N=130 (26.7%)


## 2. Sample Characteristics

In [6]:
# ═══════════════════════════════════════════════════════════════════════════════
# SECTION 3: SAMPLE OVERVIEW
# ═══════════════════════════════════════════════════════════════════════════════

print("="*70)
print("🍒 EXPERIMENT SAMPLE CHARACTERISTICS")
print("="*70)

print(f"\n📊 Sample Size: N = {len(df)}")

# Cases per item for EFA adequacy
n_items = 24
cases_per_item = len(df) / n_items
efa_status = "✓ Excellent" if cases_per_item >= 10 else "✓ Good" if cases_per_item >= 5 else "⚠️ Marginal"
print(f"   Cases per item: {cases_per_item:.1f} {efa_status}")

print("\n📊 AI Adoption:")
print(f"   Adopters: {df['AI_Adoption'].sum()} ({df['AI_Adoption'].mean()*100:.1f}%)")
print(f"   Non-Adopters: {(~df['AI_Adoption'].astype(bool)).sum()} ({(1-df['AI_Adoption'].mean())*100:.1f}%)")

print("\n📊 Usage Intensity:")
usage_dist = df['Usage_Intensity'].value_counts()
for intensity, n in usage_dist.items():
    print(f"   {intensity}: {n} ({n/len(df)*100:.1f}%)")

print("\n📊 Disability Status:")
dis_dist = df['Disability'].value_counts()
for status, n in dis_dist.items():
    print(f"   {status}: N={n} ({n/len(df)*100:.1f}%)")

print("\n" + "="*70)

🍒 EXPERIMENT SAMPLE CHARACTERISTICS

📊 Sample Size: N = 487
   Cases per item: 20.3 ✓ Excellent

📊 AI Adoption:
   Adopters: 438 (89.9%)
   Non-Adopters: 49 (10.1%)

📊 Usage Intensity:
   Medium: 161 (33.1%)
   Low: 148 (30.4%)
   High: 129 (26.5%)
   Non-User: 49 (10.1%)

📊 Disability Status:
   No: N=411 (84.4%)
   Yes: N=66 (13.6%)
   Prefer not to answer: N=10 (2.1%)



In [7]:
# ═══════════════════════════════════════════════════════════════════════════════
# SECTION 4: ITEM DESCRIPTIVES
# ═══════════════════════════════════════════════════════════════════════════════

# Define item groups
predictor_items = ['PE1', 'PE2', 'EE1', 'EE2', 'SI1', 'SI2', 'FC1', 'FC2',
                   'HM1', 'HM2', 'PV1', 'PV2', 'HB1', 'HB2', 'VO1', 'VO2',
                   'TR1', 'TR2', 'EX1', 'EX2', 'ER1', 'ER2', 'AX1', 'AX2']

outcome_items = ['BI1', 'BI2', 'BI3', 'BI4']

print("="*70)
print("ITEM DESCRIPTIVES")
print("="*70)

print("\n📊 Predictor Items (24):")
desc = df[predictor_items].describe().T[['mean', 'std', 'min', 'max']]
print(desc.round(2).to_string())

print("\n📊 Outcome Items (BI1-BI4):")
desc_bi = df[outcome_items].describe().T[['mean', 'std', 'min', 'max']]
print(desc_bi.round(2).to_string())

print("\n" + "="*70)

ITEM DESCRIPTIVES

📊 Predictor Items (24):
     mean   std  min  max
PE1  3.64  1.12  1.0  5.0
PE2  3.31  1.18  1.0  5.0
EE1  3.73  1.02  1.0  5.0
EE2  3.58  1.04  1.0  5.0
SI1  3.08  1.18  1.0  5.0
SI2  3.30  1.12  1.0  5.0
FC1  3.25  1.17  1.0  5.0
FC2  3.47  1.03  1.0  5.0
HM1  3.28  1.18  1.0  5.0
HM2  3.33  1.20  1.0  5.0
PV1  3.43  1.19  1.0  5.0
PV2  3.43  1.16  1.0  5.0
HB1  3.08  1.32  1.0  5.0
HB2  2.97  1.32  1.0  5.0
VO1  3.46  1.27  1.0  5.0
VO2  3.86  1.09  1.0  5.0
TR1  3.15  1.20  1.0  5.0
TR2  3.29  1.17  1.0  5.0
EX1  3.33  1.14  1.0  5.0
EX2  3.83  0.93  1.0  5.0
ER1  3.22  1.29  1.0  5.0
ER2  3.87  1.03  1.0  5.0
AX1  3.65  1.14  1.0  5.0
AX2  3.20  1.21  1.0  5.0

📊 Outcome Items (BI1-BI4):
     mean   std  min  max
BI1  3.22  1.19  1.0  5.0
BI2  3.21  1.22  1.0  5.0
BI3  3.26  1.24  1.0  5.0
BI4  3.06  1.24  1.0  5.0



## 3. Save Experiment Dataset

In [8]:
# ═══════════════════════════════════════════════════════════════════════════════
# SECTION 5: SAVE EXPERIMENT DATA
# ═══════════════════════════════════════════════════════════════════════════════

# Save to experiment data folder
output_path = 'data/AIRS_experiment.csv'
df.to_csv(output_path, index=False)

# Also save experiment config for reproducibility
config = {
    'selected_roles': SELECTED_ROLES,
    'roles_included': ROLES_TO_INCLUDE,
    'n_observations': len(df),
    'n_predictor_items': len(predictor_items),
    'n_outcome_items': len(outcome_items),
    'random_seed': RANDOM_SEED,
    'source': '../data/AIRS_clean.csv',
    'population_breakdown': df['Population'].value_counts().to_dict(),
    'role_breakdown': df['Role_Category'].value_counts().to_dict(),
}

with open('data/experiment_config.json', 'w') as f:
    json.dump(config, f, indent=2)

print("="*70)
print("🍒 EXPERIMENT DATA SAVED")
print("="*70)
print(f"✓ {output_path}")
print(f"  Rows: {len(df)}")
print(f"  Columns: {len(df.columns)}")
print(f"✓ data/experiment_config.json")
print("="*70)

🍒 EXPERIMENT DATA SAVED
✓ data/AIRS_experiment.csv
  Rows: 487
  Columns: 45
✓ data/experiment_config.json


## 4. Summary

In [9]:
# ═══════════════════════════════════════════════════════════════════════════════
# SECTION 6: SUMMARY
# ═══════════════════════════════════════════════════════════════════════════════

from IPython.display import display, Markdown

roles_str = ', '.join(ROLES_TO_INCLUDE)
pop_str = ', '.join([f"{k}: {v}" for k, v in df['Population'].value_counts().items()])

summary = f"""
## 🍒 Experiment Data Summary

| Attribute | Value |
|-----------|-------|
| **Sample Size** | N = {len(df)} |
| **Roles Included** | {roles_str} |
| **Population Breakdown** | {pop_str} |
| **Cases per Item** | {cases_per_item:.1f} ({efa_status}) |
| **AI Adopters** | {df['AI_Adoption'].sum()} ({df['AI_Adoption'].mean()*100:.1f}%) |

### Output Files
- `data/AIRS_experiment.csv` - Clean experiment dataset
- `data/experiment_config.json` - Configuration for reproducibility

### Next Steps
1. Run `01_EFA_Experiment.ipynb` for exploratory factor analysis
2. Run `02_CFA_Experiment.ipynb` for confirmatory factor analysis (cherry-picking)
"""

display(Markdown(summary))

print("\n" + "="*70)
print("✓ EXPERIMENT DATA PREPARATION COMPLETE")
print("="*70)


## 🍒 Experiment Data Summary

| Attribute | Value |
|-----------|-------|
| **Sample Size** | N = 487 |
| **Roles Included** | FT_Student, PT_Student, IC, Manager, Executive, Freelancer, Unemployed |
| **Population Breakdown** | Academic: 196, Professional: 161, Leader: 130 |
| **Cases per Item** | 20.3 (✓ Excellent) |
| **AI Adopters** | 438 (89.9%) |

### Output Files
- `data/AIRS_experiment.csv` - Clean experiment dataset
- `data/experiment_config.json` - Configuration for reproducibility

### Next Steps
1. Run `01_EFA_Experiment.ipynb` for exploratory factor analysis
2. Run `02_CFA_Experiment.ipynb` for confirmatory factor analysis (cherry-picking)



✓ EXPERIMENT DATA PREPARATION COMPLETE
